ПРАКТИКА 2

In [108]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
browser = webdriver.Chrome('chromedriver',chrome_options=chrome_options) # инициализируем веб драйвер


Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubun

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: use options instead of chrome_options
  if sys.path[0] == '':


In [109]:
import datetime
from time import sleep, time

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import requests
from bs4 import BeautifulSoup
import csv
from pathlib import Path

In [110]:
filename = "articles_info.csv" # имя файла, в который будем сохранять результат
base_dir= "./" # укажите директорию, в которую будем сохранять файл
user_agent = "Mozilla/5.0 ..." #Ваш user-agent. Узнать можно тут https://юзерагент.рф, смотреть с браузера Chrome
start_time = time() # время начала выполнения программы

In [111]:
def get_load_time(article_url, user_agent):
  '''load_time counter '''

  #будем ждать 3 секунды, иначе выводить exception и присваивать константное значение
  try:
      # меняем значение заголовка. По умолчанию указано, что это python-код
      headers = {
          "User-Agent": user_agent
      }
      # делаем запрос по url статьи article_url
      response = requests.get(
          article_url, headers=headers, stream=True, timeout=3.000
      )
      # получаем время загрузки страницы
      load_time = response.elapsed.total_seconds()
  except Exception as e:
      print(e)
      load_time = ">3"
  return load_time

In [112]:
def write_to_file(output_list, filename, base_dir):
    for row in output_list:
        with open(Path(base_dir).joinpath(filename), "a") as csvfile:
            fieldnames = ["id", "load_time", "rank", "points", "title", "comments", "url"]
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writerow(row)

In [113]:
def connect_to_base(browser, page_number):
    base_url = "https://news.ycombinator.com/news?p={}".format(page_number)
    for connection_attempts in range(1,4): # совершаем 3 попытки подключения
        try:
            browser.get(base_url)
            # ожидаем пока элемент table с id = 'hnmain' будет загружен на страницу
            # затем функция вернет True иначе False 
            WebDriverWait(browser, 5).until(
                EC.presence_of_element_located((By.ID, "hnmain"))
            )
            return True
        except Exception as e:
            print(e)
            print("Error connecting to {}.".format(base_url))
            print("Attempt #{}.".format(connection_attempts))
    return False

In [114]:
def parse_html(html, user_agent):
    soup = BeautifulSoup(html, "html.parser")
    output_list = []
    
    # ищем в объекте soup object id, rank, score и title статьи
    tr_blocks = soup.find_all("tr", class_="athing")
    article = 0
    for tr in tr_blocks:
        article_id = tr.get("id") # id
        article_url = tr.find_all("a")[1]["href"]

# иногда статья располагается не на внешнем сайте, а на ycombinator, тогда article_url у нее не полный, а добавочный, с параметрами. Например item?id=200933. Для этих случаев будем добавлять урл до полного
        if "item?id=" in article_url or "from?site=" in article_url:
            article_url = f"https://news.ycombinator.com/{article_url}"
        load_time = get_load_time(article_url, user_agent)
	# иногда рейтинга может не быть, поэтому воспользуемся try

        try:
            score = soup.find(id=f"score_{article_id}").string
            comments = soup.select_one(".subtext").text.split('|')[2].replace('\xa0', ' ')


        except Exception as e:
            print(e)
            score = "0 points"
            comments = "0 comments"

        article_info = {
            "id": article_id,
            "load_time": load_time,
            "rank": tr.span.string,
            "points": score,
            "title": tr.find(class_="title").string,
            "comments": comments,
            "url": article_url,
        }

        # добавляем информацию о статье в список
        output_list.append(article_info)
        article += 1
    return output_list

In [115]:
# перебираем страницы и собираем нужную информацию
for page_number in range(10):
    print("getting page " + str(page_number) + "...")
    if connect_to_base(browser, page_number):
        sleep(5)
        output_list = parse_html(browser.page_source, user_agent)
        write_to_file(output_list, filename, base_dir)

    else:
        print("Error connecting to hacker news")
    
# завершаем работу драйвера
browser.close()
sleep(1)
browser.quit()
end_time = time()
elapsed_time = end_time - start_time
print("run time: {} seconds".format(elapsed_time))

getting page 0...
getting page 1...
getting page 2...
'NoneType' object has no attribute 'string'
getting page 3...
getting page 4...
getting page 5...
getting page 6...
HTTPSConnectionPool(host='www.quantamagazine.org', port=443): Read timed out. (read timeout=3.0)
getting page 7...
HTTPSConnectionPool(host='ar5iv.labs.arxiv.org', port=443): Max retries exceeded with url: /html/1709.07020 (Caused by SSLError(SSLError("bad handshake: Error([('SSL routines', 'tls_process_server_certificate', 'certificate verify failed')])")))
HTTPSConnectionPool(host='ar5iv.labs.arxiv.org', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLError("bad handshake: Error([('SSL routines', 'tls_process_server_certificate', 'certificate verify failed')])")))
getting page 8...
getting page 9...
HTTPSConnectionPool(host='kevinboone.me', port=443): Max retries exceeded with url: /cpicom.html (Caused by SSLError(SSLError("bad handshake: Error([('SSL routines', 'tls_process_server_certificate', 'c

In [116]:
ls -l

total 92
-rw-r--r-- 1 root root 84786 Mar 30 07:19 articles_info.csv
drwxr-xr-x 1 root root  4096 Mar 23 14:22 sample_data/


In [124]:
import pandas as pd
columns = ["id", "load_time", "rank", "points", "title", "comments", "url"]
df = pd.read_csv('articles_info.csv', names = columns)
df

,id,load_time,rank,points,title,comments,url
0,30846071,0.288615,1.0,0 points,1.0,0 comments,https://www.ftc.gov/news-events/news/press-rel...
1,30851245,0.129708,2.0,0 points,2.0,0 comments,https://maps.org/2022/03/22/expanded-access-st...
2,30850625,0.072652,3.0,0 points,3.0,0 comments,https://stars.chromeexperiments.com/
3,30851371,0.11727,4.0,0 points,4.0,0 comments,https://www.happyassassin.net/posts/2016/11/04...
4,30841080,0.111044,5.0,0 points,5.0,0 comments,https://newatlas.com/energy/hb11-laser-fusion-...
...,...,...,...,...,...,...,...
775,30808945,0.139949,266.0,593 points,266.0,24 comments,https://blog.vamc19.dev/posts/dockerfile-copy-...
776,30811980,0.164459,267.0,35 points,267.0,24 comments,https://aws.amazon.com/blogs/compute/running-c...
777,30795846,0.517107,268.0,876 points,268.0,24 comments,https://blog.brixit.nl/the-end-of-the-nice-gtk...
778,30803589,0.174868,269.0,804 points,269.0,24 comments,https://aschmelyun.com/blog/i-built-a-receipt-...
